In [ ]:
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (12,6)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('axes', **{'titlesize': '16', 'labelsize': '16'})
rc('legend', **{'fontsize': '16'})
rc('figure', **{'dpi' : 150})

## Метод опорных векторов: линейные SVM

In [ ]:
def sample_mixture(N, pi, mu1, sigma1, mu2, sigma2):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    res = np.zeros((N, 2))
    res[np.where(z == 1)] = np.random.multivariate_normal(mu1, sigma1, np.sum(z))
    res[np.where(z == 0)] = np.random.multivariate_normal(mu2, sigma2, N-np.sum(z))
    return z, res

def sample_two_classes(mu1, sigma1, mu2, sigma2, pi=0.5, N=200, Ntest=None):
    z, x = sample_mixture(N, pi, mu1, sigma1, mu2, sigma2)
    if Ntest is None:
        return z, x
    else:
        test_z, test_x = sample_mixture(Ntest, pi, mu1, sigma1, mu2, sigma2)
        return z, x, test_z, test_x

def plot_points(ax, x, z, mu1=None, mu2=None):
    ax.scatter(x[np.where(z==1), 0], x[np.where(z==1), 1], marker='.', color='b')
    ax.scatter(x[np.where(z==0), 0], x[np.where(z==0), 1], marker='.', color='r')
    if mu1 is not None:
        if mu1.ndim == 1:
            ax.scatter([mu1[0]], [mu1[1]], marker='*', s=120, color='b')
        else:
            ax.scatter(mu1[:, 0], mu1[:, 1], marker='*', s=120, color='b')
    if mu2 is not None:
        if mu2.ndim == 1:
            ax.scatter([mu2[0]], [mu2[1]], marker='*', s=120, color='r')
        else:
            ax.scatter(mu2[:, 0], mu2[:, 1], marker='*', s=120, color='r')

from matplotlib import colors
cmap = colors.LinearSegmentedColormap(
    'red_blue_classes',
    {'red': [(0, 1, 1), (1, 0.7, 0.7)],
     'green': [(0, 0.7, 0.7), (1, 0.7, 0.7)],
     'blue': [(0, 0.7, 0.7), (1, 1, 1)]})

def plot_ellipse(ax, mu, sigma, color):
    v, w = sp.linalg.eigh(sigma)
    u = w[0] / sp.linalg.norm(w[0])
    angle = np.arctan(u[1] / u[0])
    angle = 180 * angle / np.pi
    ell = mpl.patches.Ellipse(mu, 2 * v[0] ** 0.5, 2 * v[1] ** 0.5,
                              180 + angle, facecolor=color,
                              edgecolor='black', linewidth=2)
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.2)
    ax.add_artist(ell)
    ax.scatter(mu[0], mu[1], marker='+', color=color, s=100)

def get_meshgrid(nx=200, ny=200):
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
    return np.meshgrid(np.linspace(x_min, x_max, nx), np.linspace(y_min, y_max, ny))

def plot_colormesh(ax, model):
    xx, yy = get_meshgrid()
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, cmap=cmap,
                   norm=colors.Normalize(0., 1.), zorder=0)
    plt.contour(xx, yy, Z, [0.5], linewidths=2., colors='white')

def plot_colormesh_svm(ax, model):
    xx, yy = get_meshgrid()
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.pcolormesh(xx, yy, Z, cmap=cmap,
                   norm=colors.Normalize(0., 1.), zorder=0)
    ax.contour(xx, yy, Z, [-1, 0, 1], linewidths=2., colors='white', linestyles=['--', '-', '--'])

def plot_svm_decision(ax, model, xx, yy):
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    ax.contour(xx, yy, Z, [0], linewidths=.5, colors='black', linestyles=['-'])

def plot_svm_decisions(ax, models):
    xx, yy = get_meshgrid()
    for model in models:
        plot_svm_decision(ax, model, xx, yy)

In [ ]:
mu1, sigma1 = np.array([-1, -.5]), .25*np.array([[1, -.5], [-.5, 2]])
mu2, sigma2 = np.array([2.5, 1]), .25*np.array([[2, -1.2], [-1.2, 2.5]])

z, x = sample_two_classes(mu1, sigma1, mu2, sigma2)

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
plot_points(ax, x, z, mu1, mu2)

plt.show()

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
plot_points(ax, x, z, mu1, mu2)

xs = np.arange(-3, 6)
ax.plot(xs, xs*-.75 + 1.2, color="black", linewidth=.75, linestyle="--")
ax.plot(xs, xs*-1. + .8, color="gray", linewidth=.5)
ax.plot(xs, xs*-20 + 3.8, color="gray", linewidth=.5)
ax.plot(xs, xs*-5 + 2.8, color="gray", linewidth=.5)
ax.plot(xs, xs*-2.52 + 1.89, color="black", linewidth=1.5)
ax.plot(xs, xs*-1.7 + 2, color="gray", linewidth=.5)
ax.plot(xs, xs*7.7 - 3.3, color="black", linewidth=.75, linestyle="--")
ax.set_xlim((-3, 5))
ax.set_ylim((-3, 3))

plt.show()

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear', C=1E100)
model.fit(x, z)

In [ ]:
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
print("%d опорных векторов" % model.support_vectors_.shape[0])
ax.scatter( model.support_vectors_[:, 0], model.support_vectors_[:, 1], color='white', s=30 )
plot_points(ax, x, z, mu1, mu2)
plot_colormesh_svm(ax, model)

plt.show()

In [ ]:
-model.coef_[0][0] / model.coef_[0][1], -model.intercept_[0] / model.coef_[0][1]

## SVM с ошибками и регуляризатором

In [ ]:
z, x = sample_two_classes(mu1, 2*sigma1, mu2, 2*sigma2)

In [ ]:
model = SVC(kernel='linear', C=1E3).fit(x, z)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
print("%d опорных векторов" % model.support_vectors_.shape[0])
ax.scatter( model.support_vectors_[:, 0], model.support_vectors_[:, 1], color='white', s=30 )
plot_points(ax, x, z, mu1, mu2)
plot_colormesh_svm(ax, model)

plt.show()

In [ ]:
z, x = sample_two_classes(mu1, 3*sigma1, mu2, 3*sigma2)
model = SVC(kernel='linear', C=1E-1).fit(x, z)

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(111)
print("%d опорных векторов" % model.support_vectors_.shape[0])
ax.scatter( model.support_vectors_[:, 0], model.support_vectors_[:, 1], color='white', s=30 )
plot_points(ax, x, z, mu1, mu2)
plot_colormesh_svm(ax, model)
plt.show()

In [ ]:
sigma_multiplier = 3
C = 1E-2
models = []
for _ in range(50):
    z, x = sample_two_classes(mu1, sigma_multiplier*sigma1, mu2, sigma_multiplier*sigma2)
    models.append( SVC(kernel='linear', C=C).fit(x, z) )

fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111)
plot_ellipse(ax, mu1, 2*sigma_multiplier*sigma1, 'r')
plot_ellipse(ax, mu2, 2*sigma_multiplier*sigma2, 'b')
# plot_points(ax, x, z, mu1, mu2)
ax.set_xlim((-2, 4))
ax.set_ylim((-2, 4))
plot_svm_decisions(ax, models)

plt.show()

## Kernel trick

In [ ]:
def sample_rings(rad_inner, rad_outer, rad_inner2, rad_outer2, pi=0.5, N=200, Ntest=None):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    rs = ( rad_inner + (rad_inner - rad_outer) * np.random.rand(np.sum(z)) )
    thetas = 2 * np.pi * np.random.rand(np.sum(z))
    rs2 = ( rad_inner2 + (rad_inner2 - rad_outer2) * np.random.rand(N - np.sum(z)) )
    thetas2 = 2 * np.pi * np.random.rand(N - np.sum(z))
    
    res = np.zeros((N, 2))
    res[np.where(z == 1)] = np.array([ rs * np.cos(thetas), rs * np.sin(thetas) ]).T
    res[np.where(z == 0)] = np.array([ rs2 * np.cos(thetas2), rs2 * np.sin(thetas2) ]).T
    return z, res

def sample_mixture(N, pi, mu1, sigma1, mu2, sigma2):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    res = np.zeros((N, 2))
    res[np.where(z == 1)] = np.random.multivariate_normal(mu1, sigma1, np.sum(z))
    res[np.where(z == 0)] = np.random.multivariate_normal(mu2, sigma2, N-np.sum(z))
    return z, res

def sample_mixtures(mu0=np.array([ [-1,-1], [1,1] ]), sigma0=2, k=5, pi=0.5, df=4, N=200, Ntest=None):
    z = np.array( np.random.rand(N) < pi, dtype=int)
    res = np.zeros((N, 2))
    mus, sigmas = [], []
    for i, n in zip(range(2), [N-np.sum(z), np.sum(z)]):
        mus.append( np.random.multivariate_normal(mu0[i], sigma0 * np.identity(2), k) )
        sigmas.append( sp.stats.invwishart.rvs(df, np.identity(2), size=k) )
        mixture_ind = np.random.randint(k, size=n)
        cur_res = np.zeros((n, 2))
        for j in range(k):
            cur_indices = np.where(mixture_ind == j)[0]
            cur_res[cur_indices] = np.random.multivariate_normal(mus[-1][j], sigmas[-1][j], len(cur_indices))
        res[np.where(z == i)] = cur_res
    return z, res, mus, sigmas

In [ ]:
z, x = sample_rings(0, 3, 4, 6)

model = SVC(kernel='linear', C=1E-1).fit(x, z)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
plot_points(ax, x, z)
plot_colormesh_svm(ax, model)
plt.show()

In [ ]:
C = 1e1


model = SVC(kernel='poly', degree=2, C=C).fit(x, z)

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
print("%d опорных векторов" % model.support_vectors_.shape[0])
ax.scatter( model.support_vectors_[:, 0], model.support_vectors_[:, 1], color='white', s=40 )
plot_points(ax, x, z)
plot_colormesh_svm(ax, model)
plt.show()

In [ ]:
z, x, mus, sigmas = sample_mixtures(sigma0=5, N=500)

In [ ]:
C = 1e1
model = SVC(kernel='rbf', C=C).fit(x, z)

fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(111)
print("%d опорных векторов" % model.support_vectors_.shape[0])
ax.scatter( model.support_vectors_[:, 0], model.support_vectors_[:, 1], color='white', s=40 )
plot_points(ax, x, z, mu1=mus[1], mu2=mus[0])
plot_colormesh_svm(ax, model)
plt.show()